<a href="https://colab.research.google.com/github/mmsamiei/thesis-prototype/blob/master/phase2_littlebert_v1.ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install transformers

     |████████████████████████████████| 450kB 4.8MB/s 
     |████████████████████████████████| 870kB 72.5MB/s 
     |████████████████████████████████| 1.0MB 54.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884629 sha256=41082981ff38674fe275d8af700fe49322799d2287f26a0e69a5964648352f97
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
!cp /content/drive/My\ Drive/Thesis/phase-2/train_tokenized_bert.json ./train.json

In [0]:
from torch.utils.data import Dataset, DataLoader
import os
import torch
import json
from torch.utils.data.sampler import SubsetRandomSampler

In [0]:
class MyDataset(Dataset):
    """My dataset."""

    def __init__(self, json_file):
        """
        Args:
            json_file (string): Path to the json file with annotations.
        """
        self.dialogues = json.load(open(json_file))
        self.root_dir = json_file

        self.dialogues = sorted(self.dialogues, key=lambda x: len(x['history']))

    def __len__(self):
        return len(self.dialogues)

    def __getitem__(self, idx):
      
        
        history_lst = self.dialogues[idx]['history']
        true_lst = self.dialogues[idx]['true_sentence']
        false_lst = self.dialogues[idx]['false_sentenc']


        if(len(history_lst)>40):
          history_lst = history_lst[-40:]

        if(len(true_lst)>40):
          true_lst = true_lst[:40]
        
        if(len(false_lst)>40):
          false_lst = false_lst[:40]
        

        history = torch.LongTensor(history_lst)
        true_sample = torch.LongTensor(true_lst)
        false_sample = torch.LongTensor(false_lst)

        sample = {'history': history, 'true_sample': true_sample, 'false_sample': false_sample}

        return sample

In [0]:
dataset = MyDataset('train.json')

In [0]:
def my_collate_fn(batch):

  len_batch = len(batch)

  
  max_len_history = max([len(data['history']) for data in batch])
  max_len_true_sample = max([len(data['true_sample']) for data in batch])
  max_len_false_sample = max([len(data['false_sample']) for data in batch])
  
  padding_ind = 1 ## for bert is 0 but because embedding hate 0!! we used 1
  result_history = torch.ones(len_batch, max_len_history)
  result_true_sample = torch.ones(len_batch, max_len_true_sample)
  result_false_sample = torch.ones(len_batch, max_len_false_sample)

  for i, data in enumerate(batch):
    p1 = len(data['history'])
    result_history[i, :p1] = data['history']
    p2 = len(data['true_sample'])
    result_true_sample[i, :p2] = data['true_sample']
    p3 = len(data['false_sample'])
    result_false_sample[i, :p3] = data['false_sample']



  return result_history.long(), result_true_sample.long(), result_false_sample.long()

sampler = torch.utils.data.SequentialSampler(dataset)
dataset_loader = torch.utils.data.DataLoader(dataset, batch_size=256, sampler=sampler,
                                             shuffle=False, collate_fn=my_collate_fn)

In [0]:
for _, batch in enumerate(dataset_loader):
  history, true_sample, false_sample = batch

# Model

In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [0]:
import torch
from transformers import *
import numpy as np
import torch.nn as nn

class RepresentModel(nn.Module):
  
  def __init__(self, device):
    super().__init__()

    self.device = device
    
    self.bert = BertModel.from_pretrained('bert-base-uncased')
    del self.bert.encoder.layer[1:]

    for p in self.bert.encoder.layer[0:1].parameters():
      p.requires_grad = False

    for p in self.bert.embeddings.parameters():
      p.requires_grad = False


    self.fc = nn.Linear(768, 128)

  def forward(self, x):
    ## x = [batch, sent]
    batch_size, sent_len = x.shape[0], x.shape[1]

    temp = self.bert(x)
    temp = temp[0]
    ## [batch, sent_len, 768]
    temp = temp[:,0,:]
    ## [batch, 768]
    temp = self.fc(temp)
    return temp


In [0]:
hid_size = 768
model = RepresentModel(device).to(device)

In [73]:
test_len = 20
batch_size = 64
vocab_size = 30000
test_input = torch.LongTensor(batch_size, test_len).random_(1,vocab_size).to(device)
model(test_input).shape

torch.Size([64, 128])

In [74]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 689,024 trainable parameters


In [75]:
model.bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [76]:
for p in model.named_parameters():
  print(p[0],"\t",p[1].requires_grad)

bert.embeddings.word_embeddings.weight 	 False
bert.embeddings.position_embeddings.weight 	 False
bert.embeddings.token_type_embeddings.weight 	 False
bert.embeddings.LayerNorm.weight 	 False
bert.embeddings.LayerNorm.bias 	 False
bert.encoder.layer.0.attention.self.query.weight 	 False
bert.encoder.layer.0.attention.self.query.bias 	 False
bert.encoder.layer.0.attention.self.key.weight 	 False
bert.encoder.layer.0.attention.self.key.bias 	 False
bert.encoder.layer.0.attention.self.value.weight 	 False
bert.encoder.layer.0.attention.self.value.bias 	 False
bert.encoder.layer.0.attention.output.dense.weight 	 False
bert.encoder.layer.0.attention.output.dense.bias 	 False
bert.encoder.layer.0.attention.output.LayerNorm.weight 	 False
bert.encoder.layer.0.attention.output.LayerNorm.bias 	 False
bert.encoder.layer.0.intermediate.dense.weight 	 False
bert.encoder.layer.0.intermediate.dense.bias 	 False
bert.encoder.layer.0.output.dense.weight 	 False
bert.encoder.layer.0.output.dense.bias 	

In [0]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
    
    def zero_grad(self):
        self.optimizer.zero_grad()

In [78]:
!pip install -U tqdm

     |████████████████████████████████| 61kB 3.2MB/s 
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
from tqdm.notebook import tqdm

def train_one_epoch(model,train_iter, optimizer, criterion, clip):
  epoch_loss = 0
  model.train()
  for batch in tqdm(train_iter):
    optimizer.zero_grad()
    history, true_sample, false_sample = batch
    batch_size = history.shape[0]
    ### hist = [batch, sent_len]
    history_rpr = model(history.to(device))
    true_rpr = model(true_sample.to(device))
    false_rpr = model(false_sample.to(device))
    ## rpr = [batch_size, hidden]
    cos = nn.PairwiseDistance().to(device)
    tru_sml = -1*cos(history_rpr, true_rpr)
    fls_sml = -1*cos(history_rpr, false_rpr)
    mini_batch_tensor = torch.ones(batch_size).to(device)
    loss = criterion(tru_sml, fls_sml, mini_batch_tensor)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    epoch_loss += loss.item()
  return epoch_loss / len(train_iter)

In [0]:
def train(model, train_iter, optimizer, criterion, clip, N_EPOCH):
  for epoch in range(N_EPOCH):
    epoch_loss = train_one_epoch(model, train_iter, optimizer, criterion, clip)
    print(epoch_loss)

In [95]:
optimizer = NoamOpt(hid_size, 1, 2000,
              torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
criterion = torch.nn.MarginRankingLoss(margin=10).to(device)
train(model, dataset_loader, optimizer, criterion, 1, 1)


0.2911392762595191


# Evaluate

In [0]:
def evaluate(model,valid_iter, criterion):
  epoch_loss = 0
  model.eval()
  for batch in tqdm(valid_iter):
    history, true_sample, false_sample = batch
    batch_size = history.shape[0]
    ### hist = [batch, sent_len]
    history_rpr = model(history.to(device))
    true_rpr = model(true_sample.to(device))
    false_rpr = model(false_sample.to(device))
    ## rpr = [batch_size, hidden]
    cos = nn.PairwiseDistance().to(device)
    tru_sml = -1*cos(history_rpr, true_rpr)
    fls_sml = -1*cos(history_rpr, false_rpr)
    mini_batch_tensor = torch.ones(batch_size).to(device)
    loss = criterion(tru_sml, fls_sml, mini_batch_tensor)
    epoch_loss += loss.item()
  return epoch_loss / len(valid_iter)

In [0]:
valid_dataset = MyDataset('/content/drive/My Drive/Thesis/phase-2/valid_tokenized_bert.json')
valid_dataset_sampler = torch.utils.data.SequentialSampler(valid_dataset)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=128, sampler=valid_dataset_sampler,
                                             shuffle=False, collate_fn=my_collate_fn)

In [98]:
evaluate(model,valid_loader, criterion)

0.30323302276870784

In [0]:
torch.save(model.state_dict(), 'phase2_little_bert_v1_epoch1.model')

In [0]:
!cp /content/phase2_little_bert_v1_epoch1.model /content/drive/My\ Drive/Thesis/phase-2

In [0]:
model.state_dict()

OrderedDict([('bert.embeddings.word_embeddings.weight',
              tensor([[-0.0102, -0.0615, -0.0265,  ..., -0.0199, -0.0372, -0.0098],
                      [-0.0117, -0.0600, -0.0323,  ..., -0.0168, -0.0401, -0.0107],
                      [-0.0198, -0.0627, -0.0326,  ..., -0.0165, -0.0420, -0.0032],
                      ...,
                      [-0.0218, -0.0556, -0.0135,  ..., -0.0043, -0.0151, -0.0249],
                      [-0.0462, -0.0565, -0.0019,  ...,  0.0157, -0.0139, -0.0095],
                      [ 0.0015, -0.0821, -0.0160,  ..., -0.0081, -0.0475,  0.0753]],
                     device='cuda:0')),
             ('bert.embeddings.position_embeddings.weight',
              tensor([[ 1.7505e-02, -2.5631e-02, -3.6642e-02,  ...,  3.3437e-05,
                        6.8312e-04,  1.5441e-02],
                      [ 7.7580e-03,  2.2613e-03, -1.9444e-02,  ...,  2.8910e-02,
                        2.9753e-02, -5.3247e-03],
                      [-1.1287e-02, -1.9644e-03, -